# Assignment 3: Creating a WordyPy Solver

You now understand the rules of _WordyPy_ and how to read the game state from an image. Now your job is to read in a
_WordyPy_ partial play and provide a next **good** guess. What's a good guess? A good guess is one which:

1. Continues to adhere to the rules of _WordPy_
2. Does not repeat words which have already been played
3. Uses the knowledge of previous guesses to pick a new good word

Unlike previous assignments there are no guardrails for this task -- you can complete this using whatever software
architecture you like! In addition, I've put my code in a new module for you to import, called `wordy`. You can just
import this module and begin making calls to it. You should be able to understand how it works by reading the module
documentation.


In [6]:
import wordy
import PIL
import pytesseract
import random
pytesseract.pytesseract.tesseract_cmd = r'PATH TO Tesseract'


def solution(board: PIL.Image) -> str:
    """The student solution to the problem.

    You must write code to query the wordy module and make
    a guess for the word. You needs to inspect the module to
    understand how to do this, and this function should only return
    the guess that you are going to make based on the game board state.

    Returns:
        str: The guess that you are going to make.
    """
    word_list = wordy.get_word_list()
    display_spec = wordy.get_display_spec()
    correct_letters = {}
    misplaced_letters = set()
    excluded_letters = set()

    # Get image dimensions
    img_width, img_height = board.size

    # Step 1: Analyze the board to extract clues
    for i in range(5):  # Assume 5 rows for guesses
        y = i * display_spec.block_height
        
        if y >= img_height:  # Prevent out-of-bounds access
            break

        for j in range(5):  # Assume 5 letters per guess
            x = j * (display_spec.block_width + display_spec.space_between_letters)
            
            if x >= img_width:  # Prevent out-of-bounds access
                break

            # Ensure the pixel is within the image bounds
            if (x + display_spec.block_width - 5 < img_width) and (y + display_spec.block_height - 5 < img_height):
                # Get the color of the block surrounding the letter
                color_block = board.getpixel((x + display_spec.block_width - 5, y + display_spec.block_height - 5))
                
                # Convert hex colors to RGB tuples for comparison
                correct_color = tuple(int(display_spec.correct_location_color[i:i+2], 16) for i in (1, 3, 5))
                misplaced_color = tuple(int(display_spec.incorrect_location_color[i:i+2], 16) for i in (1, 3, 5))
                incorrect_color = tuple(int(display_spec.incorrect_color[i:i+2], 16) for i in (1, 3, 5))

                # Crop the area where the letter appears
                letter_box = (x, y, x + display_spec.block_width, y + display_spec.block_height)
                letter_image = board.crop(letter_box)

                # Use Tesseract OCR to extract the letter
                letter = pytesseract.image_to_string(letter_image, config='--psm 10').strip()

                # Determine the letter's status based on the block color
                if color_block == correct_color:
                    correct_letters[j] = letter  # Correct letter in the correct place
                elif color_block == misplaced_color:
                    misplaced_letters.add(letter)  # Correct letter, wrong place
                elif color_block == incorrect_color:
                    excluded_letters.add(letter)  # Incorrect letter

    # Step 2: Generate a new guess based on the clues
    for word in word_list:
        # Check if all known correct letters are in the right place
        if all(word[position] == letter for position, letter in correct_letters.items()) and \
           all(letter in word for letter in misplaced_letters) and \
           all(letter not in word for letter in excluded_letters):
            return word

    # Return any word if no guess fits the criteria
    return random.choice(word_list)

# wordy.set_debug_mode(debug=True)
# wordy.get_board_state(target_word_debug="ANNEX", guess_words_debug=["APAID", "BOUGE", "CABLE"])

In [7]:
# The autograder for this assignment is easy, it will try and play
# a few rounds of the game and ensure that errors are not thrown. If
# you can make it through five rounds we'll assume you have the right
# solution!
#
# You SHOULD NOT change anything in the wordy module, instead you
# must figure out how to write the solution() function in this notebook
# to make a good guess based on the board state!

for i in range(5):
    try:
        # Get an image of the current board state from wordy.
        # Note that the image contains some number of random guesses (always less than 5 guesses).
        image = wordy.get_board_state()
        # Create a new *good* guess based on the image and rules of wordy
        new_guess = solution(image)  # your code goes in solution()!
        # Send that guess to wordy to make sure it doesn't throw any errors
        wordy.make_guess(new_guess)
    except Exception as e:
        raise e

TesseractError: (2, 'Usage: pytesseract [-l lang] input_file')